In [1]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [51]:
from datetime import datetime, timedelta
from pandas import DataFrame, to_datetime, set_option
from TpsParse.Tps.TpsFile import TpsFile
import os
from numpy import quantile, nan
from math import ceil

set_option('display.max_columns', 500)
set_option('display.max_row', 1000)

from bokeh.resources import CDN
from bokeh.embed import components
from bokeh.models import Range1d, NumeralTickFormatter, ColumnDataSource, FactorRange, HoverTool, \
DataRange1d, DatetimeTickFormatter
from bokeh.plotting import figure
from bokeh.transform import linear_cmap, factor_cmap
from bokeh.io import show, output_notebook
from bokeh import palettes

from django.db.models import Sum, DateField
from django.db.models.functions import Trunc
from Utils.clean_string import clean_string

output_notebook()

Loading BokehJS ...

In [3]:
def parser(path):
    file = open(path, "r+b")
    tps = TpsFile(file)

    cols_and_type = []
    columns = []
    records = []
    for definition in tps.get_table_definitions():
        for field in definition.fields:
            column = field.field_name.split(':')[1].lower()
            col_and_type = (field.field_name.split(':')[1].lower(), field.type)
            if column not in columns:
                columns.append(column)
                cols_and_type.append(col_and_type)
        for record in tps.get_data_records(definition):
            records.append([r.strip().title() if isinstance(r, str) else r for r in record.values])

    table = []
    for record in records:
        dic = {}
        for col, rec in zip(columns, record):
            dic[col] = rec
        table.append(dic)

    df = DataFrame(table, columns=columns)

    name_list = ('fec', 'periodo')
    # Formatear la fecha.
    date_columns = [cname for cname, ctype in cols_and_type if cname.startswith(name_list) and ctype == 'SignedLong']
    for col in df[date_columns]:
        df[col] = datetime(1800, 12, 28) + df[col].map(timedelta)
        df.loc[df[col] == '1800-12-28', col] = None

    # Reemplazar los campos vacíos con None.
    df.replace(to_replace='', value=nan, inplace=True)
    df.replace(to_replace=0, value=None, inplace=True)
    
    # Borrar los registros que tengan todos valores nulos.
    df.dropna(axis='index', how='all', inplace=True)
    
    # Eliminar registros duplicados.
    df.drop_duplicates(keep='last', inplace=True)
    
    # Borrar las columnas que tengan todos valores nulos o ceros.
    df.dropna(axis='columns', how='all', inplace=True)
    df = df.loc[:, (df != 0).any(axis=0)]
    
    return df

In [5]:
path = "/home/wen/Documentos/Proyecto/Datos Agrosoft/Cli.Tps"
df = parser(path);df

,codigo,razon_social,direccion,iva,cuit,localidad,prefijo,tipo_factura,tipo_cliente,saldo,contacto,linea1,linea2,linea3,celular,particip_total_compras,porcent_total_compras,nombre_fantasia
0,2,Hèctor Delbino,NaN,Inscripto,20063078604,2313*,1,1,25,0.00,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN
1,3,Coop Agrìc Ganad Bdo Irigoyen,Juan Xxiii 128,Inscripto,30508118690,2248,0,1,25,309734.55,NaN,03466-494025,492032,494323,NaN,134948.49,6.01,NaN
2,4,Franco C Morra,Moreno 282,Inscripto,20248607506,2309,0,1,25,0.00,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN
3,5,Josè M Morra,Moreno 282,Inscripto,20063014797,2309,0,1,25,0.00,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN
4,6,Tradiciòn Ganadera Sa,NaN,Inscripto,30708736895,NaN,0,1,25,0.00,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN
5,7,Rosa A Cipolatti,Italia 1346,Inscripto,27042840799,2421,1,1,25,0.00,NaN,NaN,NaN,NaN,NaN,24310.00,1.08,NaN
6,8,Paira Maria Mercedes Y Nestor,Zona Rural,Inscripto,30625512456,3029,1,1,25,0.00,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN
7,9,Coop.Agric.Gan.Guillrmo Lehman,San Martin 1531,Inscripto,30508273181,1629,0,1,27,0.00,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN
8,10,Hugo Pedro Carreras,NaN,Inscripto,20127942146,NaN,0,1,25,0.00,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN
9,11,Ecoagro Srl,NaN,Inscripto,30712319999,NaN,0,1,25,0.00,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN


In [20]:
from app.models import Cli

cli = Cli.objects.values(
        'razon_social',
        'iva',
        'saldo',
        'prefijo',
        'tipo_factura',
        'tipo_cliente',
        'particip_total_compras',
        'porcent_total_compras')

df = DataFrame(cli)
df.sort_values('saldo', inplace=True)

In [75]:
df_mayor = df[df.saldo > 0].copy()
source_mayor = ColumnDataSource(df_mayor)
df_minor = df[df.saldo < 0].copy()
source_minor = ColumnDataSource(df_minor)

In [78]:
source_mayor = ColumnDataSource(df_mayor)
# source_mayor.data['color'] = palettes.Greens
tooltips = [("Saldo", "@saldo{$0.00}"),
                ("Participación total de compras",
                 "@particip_total_compras{$0.00}"),
                ("Porcentaje total de compras", "@porcent_total_compras{0%}"),
                ("Tipo de cliente", "@tipo_cliente"),
                ("Iva", "@iva")]
hovertool = HoverTool(mode='hline', attachment='below',
                      names=['hbar'], tooltips=tooltips)
plot_mayor = figure(
    title="Clientes con saldo mayor a cero",
    y_range=FactorRange(*source_mayor.data['razon_social'],
                        bounds='auto'),
    x_range=Range1d(start=0,
                    end=(max(source_mayor.data['saldo']) +
                         max(source_mayor.data['saldo']) * 0.2),
                    bounds='auto'),
    tools=["tap,reset,save", hovertool],
    toolbar_location='above', 
    y_axis_label='Cliente', x_axis_label='Saldo')
plot_mayor.hbar(
    y='razon_social', right='saldo', height=0.8, source=source_mayor,
#     fill_color='color', 
    nonselection_fill_alpha=0.2,
    line_color=None, name='hbar')
show(plot_mayor)

In [54]:
source_mayor.data

{'index': array([ 0, 69, 68, 66, 65, 64, 62, 61, 60, 59, 58, 57, 55, 54, 53, 52, 51,
        50, 97, 48, 70, 47, 71, 73, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87,
        86, 85, 84, 83, 81, 77, 76, 75, 74, 72, 46, 49, 44, 23, 22, 45, 20,
        19, 18, 17, 16, 15,  3, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5, 25,
        24, 26, 43, 42, 41, 40, 39, 38, 37, 36, 35,  2, 34, 33, 32, 31, 30,
        29, 28, 27,  4, 78, 63, 80, 56, 79, 21, 67, 82,  1, 98]),
 'razon_social': array(['Hèctor Delbino', 'Ruben Sereno', 'Matadero Frigorìfico Uniòn Sa',
        'Lozano Anelo Tesio', 'Allione Rogelio Alcides',
        'Morra Jose Manuel', 'Rossetti Alberto Eduardo',
        'Odetti Eladio Y Odetti German', 'Fiorito Y Sebastiani S.A.',
        'Giorgis Rudecindo', 'Brignone Jose Alberto',
        'Civallero Ruben Honorio', 'Sale Lucas Rene',
        'Mandrille Elena Cristina', 'Colombatto Maria J.',
        'Correnti Mario Y Jorge', 'Soldano Jose Luis', None,
        'Racca Jose Luis', 'Fiderovich M. 